In [1437]:
#------- EXERCICI 1 --- ACABAT
import pandas as pd

data = pd.read_csv('DelayedFlights.csv')

df = data.sample(n=200000, replace=False)

df = df[[
    #'Unnamed: 0', 'Year', 'Month', 'DayofMonth',
    'DayOfWeek',
    #'DepTime',
    #'CRSDepTime', 'ArrTime', 'CRSArrTime',
    'UniqueCarrier', 'FlightNum', 'TailNum', 'ActualElapsedTime',
    #'CRSElapsedTime',
    'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 'Distance', 
    #'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
    #'WeatherDelay', 'NASDelay', 'SecurityDelay',
    #'LateAircraftDelay'
    ]].copy()

En este dataset podemos encontrar información sobre los vuelos de 20 aerolíneas comerciales en los Estados Unidos en 2008. Ésta muestra del dataset contiene 30 columnas con datos sobre 200.000 vuelos tanto antes de realizarse, así como tras la finalización de los mismos. De esta manera, por ejemplo, podemos obtener información sobre retrasos, desviaciones o cancelaciones y sus causas.

Puesto que no sabría extraer información de ello, podría eliminar las dos columnas que hacen referencia a la bajada y la subida del tren de aterrizaje (TaxiIn y TaxiOut).

Por otro lado, tambien puedo eliminar todas las columnas que se refieren a los retrasos y las cancelaciones según la causa, así como las columnas sobre la fecha como "Year", "Month" o "DayOfMonth", como también aquellas relacionadas con la programación original del vuelo como "CRSDepTime" o "CRSElapsedTime", ya que tampoco haré uso de estas columnas para los ejercicios que siguen.

Finalmente, modifico el nombre de las columnas para facilitar la lectura y vuelvo a indexar los registros para darles un valor entre 0 y 200.000 acorde con la totalidad de la muestra y no con la base de datos original.

In [1438]:
df = df.rename(columns = {'DayOfWeek':'Day_Of_Week',
                          'UniqueCarrier':'Carrier',
                          'FlightNum':'Flight_Number',
                          'TailNum':'Tail_Number',
                          'ActualElapsedTime':'Actual_Elapsed_Time',
                          'AirTime':'Air_Time',
                          'ArrDelay':'Arrival_Delay',
                          'DepDelay':'Departure_Delay',
                          'Dest':'Destination'}) \
    .reset_index(drop=True)

In [1439]:
df.describe()

,Day_Of_Week,Flight_Number,Actual_Elapsed_Time,Air_Time,Arrival_Delay,Departure_Delay,Distance
count,200000.000000,200000.000000,199148.000000,199148.000000,199148.000000,200000.000000,200000.00000
mean,3.983605,2188.975325,133.429143,108.408897,42.194905,43.149355,766.74979
std,1.993328,1947.855025,72.205026,68.751976,57.214460,53.771353,575.17129
min,1.000000,1.000000,15.000000,0.000000,-67.000000,6.000000,30.00000
25%,2.000000,612.000000,80.000000,58.000000,9.000000,12.000000,340.00000
50%,4.000000,1545.000000,116.000000,90.000000,24.000000,24.000000,606.00000
75%,6.000000,3441.000000,165.000000,137.000000,56.000000,53.000000,998.00000
max,7.000000,9741.000000,727.000000,645.000000,2453.000000,2467.000000,4962.00000


Al ejecutar la función describe() podemos ver que faltan algunos datos en al menos 3 columnas, por lo que tendremos que volver a ellas más adelante.

Por otro lado, también podemos observar que el valor mínimo de la columna 'Arrival_Delay' es negativo. Esto es normal, ya que en esta misma columna se expresa mediante un valor negativo cuando el vuelo llega antes de lo programado.

Sin embargo, el valor máximo de la misma columna de 'Arrival_Delay' parece demasiado elevado en comparación con el 75% más bajo, que a penas llega a la hora de retraso, así que acto seguido miraremos si sigue una progresión normal.

Finalmente, también podemos observar que el valor mínimo de la columna 'Distance' parece muy bajo para un trayecto en avión, por lo que podría ser un error a primera vista, sin emabrgo al ver la cantidad de vuelos por distancia vemos que es normal que haya trayectos así de cortos.

In [1440]:
df_more_delayed = df.loc[df['Distance'] < 50]
df_more_delayed.sort_values(by='Distance', ascending=True)
#df['Distance'].plot(kind='hist', bins=100)

,Day_Of_Week,Carrier,Flight_Number,Tail_Number,Actual_Elapsed_Time,Air_Time,Arrival_Delay,Departure_Delay,Origin,Destination,Distance
15733,5,OH,5052,N442CA,74.0,23.0,22.0,8.0,HPN,JFK,30
198715,7,AS,65,N775AS,38.0,9.0,28.0,17.0,WRG,PSG,31
92547,7,AS,64,N763AS,29.0,19.0,36.0,34.0,PSG,WRG,31
173777,1,AS,64,N764AS,20.0,11.0,8.0,15.0,PSG,WRG,31
98619,7,AS,65,N791AS,24.0,15.0,60.0,60.0,WRG,PSG,31
...,...,...,...,...,...,...,...,...,...,...,...
143965,1,OO,5458,N221SW,32.0,22.0,48.0,58.0,OXR,LAX,49
146333,4,OO,5512,N584SW,33.0,15.0,11.0,19.0,OXR,LAX,49
149251,3,OO,5457,N295UX,34.0,18.0,224.0,233.0,OXR,LAX,49
98761,1,OO,5512,N233SW,31.0,18.0,-5.0,8.0,OXR,LAX,49


De vuelta al valor máximo de la misma columna de 'Arrival_Delay', al consultar los registros según el mayor tiempo de retraso podemos observar que parece disminuir de manera normal y que son pocos los vuelos que acumulan mucho retraso.

In [1441]:
df_more_delayed = df.loc[df['Arrival_Delay'] > 1000]
df_more_delayed.sort_values(by='Arrival_Delay', ascending=False)[:]
#df['Arrival_Delay'].plot(kind='hist', bins=300)

,Day_Of_Week,Carrier,Flight_Number,Tail_Number,Actual_Elapsed_Time,Air_Time,Arrival_Delay,Departure_Delay,Origin,Destination,Distance
14502,4,NW,1699,N3324L,154.0,132.0,2453.0,2467.0,CLT,MSP,930
100830,5,MQ,3538,N512AE,72.0,50.0,1707.0,1710.0,LIT,DFW,304
34241,4,NW,1765,N600TR,137.0,72.0,1392.0,1369.0,PHL,DTW,453
158962,4,NW,624,N376NW,145.0,124.0,1359.0,1367.0,ABQ,MSP,981
181727,3,UA,68,N548UA,336.0,315.0,1322.0,1303.0,LIH,LAX,2615
47416,3,AA,1138,N610AA,319.0,124.0,1312.0,1153.0,LAS,DFW,1055
84715,3,AA,1598,N5ELAA,291.0,128.0,1298.0,1167.0,LAS,DFW,1055
109840,6,NW,590,N556NW,184.0,165.0,1249.0,1260.0,PDX,MSP,1426
162604,2,NW,1077,N987US,170.0,123.0,1203.0,1182.0,TPA,IND,838
116327,3,NW,940,N354NB,153.0,128.0,1166.0,1167.0,FCA,MSP,1026


In [1442]:
df.dtypes

Day_Of_Week              int64
Carrier                 object
Flight_Number            int64
Tail_Number             object
Actual_Elapsed_Time    float64
Air_Time               float64
Arrival_Delay          float64
Departure_Delay        float64
Origin                  object
Destination             object
Distance                 int64
dtype: object

Por último vemos que el tipo de dato tiene sentido con lo que representan, así que los dejaremos sin modificar.

In [1443]:
#------- EXERCICI 2 --- ACABAT
### Troba quantes dades faltants hi ha per columna.
print('Los registros que faltan por columna son:')
pd.isnull(df).sum()

Los registros que faltan por columna son:


Day_Of_Week              0
Carrier                  0
Flight_Number            0
Tail_Number              0
Actual_Elapsed_Time    852
Air_Time               852
Arrival_Delay          852
Departure_Delay          0
Origin                   0
Destination              0
Distance                 0
dtype: int64

In [1444]:
#------- EXERCICI 2 --- ACABAT
### Depuración: Rellenar Na con la media.
df[['Actual_Elapsed_Time', 'Air_Time', 'Arrival_Delay']] = df[['Actual_Elapsed_Time', 'Air_Time', 'Arrival_Delay']].fillna(df[['Actual_Elapsed_Time', 'Air_Time', 'Arrival_Delay']].mean())
#df = df.loc[~df['Arrival_Delay'].isna()]
print(df.isna().sum())

Day_Of_Week            0
Carrier                0
Flight_Number          0
Tail_Number            0
Actual_Elapsed_Time    0
Air_Time               0
Arrival_Delay          0
Departure_Delay        0
Origin                 0
Destination            0
Distance               0
dtype: int64


In [1445]:
df.describe()

,Day_Of_Week,Flight_Number,Actual_Elapsed_Time,Air_Time,Arrival_Delay,Departure_Delay,Distance
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000
mean,3.983605,2188.975325,133.429143,108.408897,42.194905,43.149355,766.74979
std,1.993328,1947.855025,72.051064,68.605377,57.092462,53.771353,575.17129
min,1.000000,1.000000,15.000000,0.000000,-67.000000,6.000000,30.00000
25%,2.000000,612.000000,80.000000,58.000000,9.000000,12.000000,340.00000
50%,4.000000,1545.000000,116.000000,91.000000,25.000000,24.000000,606.00000
75%,6.000000,3441.000000,165.000000,137.000000,55.000000,53.000000,998.00000
max,7.000000,9741.000000,727.000000,645.000000,2453.000000,2467.000000,4962.00000


In [1446]:
#------- EXERCICI 2 --- ACABAT
### Haz un análisis estadístico que consideres relevante.
df1 = df[['Day_Of_Week', 'Arrival_Delay']]
df1_DayOfWeek_Delay = df1.groupby('Day_Of_Week').aggregate(sum)
df1_DayOfWeek_Delay.sort_values(by='Arrival_Delay', ascending=False)

,Arrival_Delay
Day_Of_Week,
5,1.464159e+06
7,1.323926e+06
1,1.250554e+06
4,1.239228e+06
2,1.171909e+06
3,1.093850e+06
6,8.953539e+05


En esta nuevo data frame podemos ver que el mayor número de retrasos suceden los viernes (5) y los domingos (7) posiblemente debido a las idas y las vueltas del fin de semana, y que por la misma razón que el menor número de retrasos se dan en sábado (6).

In [1447]:
#------- EXERCICI 2 --- ACABAT
### Crear columna velocidad mediana de los vuelos, si han llegado tarde o no...
df2 = df[['Flight_Number', 'Air_Time', 'Distance', 'Arrival_Delay', 'Departure_Delay']]
df2["Mean_Speed(m/m)"] = df2['Distance'] / df2['Air_Time']
df2['Delayed'] = df2['Arrival_Delay'].apply(lambda x:'Yes' if x > 0 else 'No')
df2.head()

C:\Users\dcuxa\AppData\Local\Temp\ipykernel_32632\2382179659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Mean_Speed(m/m)"] = df2['Distance'] / df2['Air_Time']
C:\Users\dcuxa\AppData\Local\Temp\ipykernel_32632\2382179659.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Delayed'] = df2['Arrival_Delay'].apply(lambda x:'Yes' if x > 0 else 'No')


,Flight_Number,Air_Time,Distance,Arrival_Delay,Departure_Delay,Mean_Speed(m/m),Delayed
0,679,128.0,733,138.0,92.0,5.726562,Yes
1,190,45.0,261,13.0,6.0,5.800000,Yes
2,166,65.0,345,3.0,7.0,5.307692,Yes
3,2871,133.0,883,42.0,44.0,6.639098,Yes
4,3319,148.0,765,79.0,58.0,5.168919,Yes


In [1448]:
#------- EXERCICI 2 --- ACABAT
### Fes una taula de les aerolínies amb més endarreriments acumulats.
df3 = df[['Carrier', 'Arrival_Delay']]
df3_UniqueCarrier_ArrDelay = df3.groupby('Carrier').aggregate(sum)
df3_UniqueCarrier_ArrDelay.sort_values(by='Arrival_Delay', ascending=False)

,Arrival_Delay
Carrier,
WN,1.173712e+06
AA,9.384280e+05
UA,6.983549e+05
MQ,6.575386e+05
OO,6.260139e+05
XE,5.316304e+05
DL,4.565175e+05
CO,4.141112e+05
EV,4.076703e+05


In [1449]:
#------- EXERCICI 2 --- ACABAT
### ¿Cuáles son los vuelos más largos?
df4 = df[['Flight_Number', 'Air_Time']]
df4_more_airtime = df4.groupby('Flight_Number').aggregate('max')
df4_more_airtime.sort_values(by='Air_Time', ascending=False)[:5]

,Air_Time
Flight_Number,
15,645.0
851,592.0
1273,583.0
1561,559.0
14,558.0


In [1450]:
#------- EXERCICI 2 --- ACABAT
### I els més endarrerits?
df5 = df[['Flight_Number', 'Arrival_Delay']]
df5_more_delayed = df5.groupby('Flight_Number').aggregate('max')
df5_more_delayed.sort_values(by='Arrival_Delay', ascending=False)[:5]

,Arrival_Delay
Flight_Number,
1699,2453.0
3538,1707.0
1765,1392.0
624,1359.0
68,1322.0


In [1451]:
#------- EXERCICI 2 --- ACABAT
### Busca les rutes mes llarges.
df6 = df[['Distance', 'Origin', 'Destination']]
df6_longest_routes = df6.groupby(['Origin', 'Destination']).aggregate('max')
df6_longest_routes.sort_values(by=['Distance'], ascending=False)[:5]

Distance
Origin Destination          
EWR    HNL              4962
HNL    EWR              4962
       ATL              4502
ATL    HNL              4502
ORD    HNL              4243

In [1452]:
#------- EXERCICI 2 --- ACABAT
### ... y las rutas que acumulan más retrasos.
df7 = df[['Arrival_Delay', 'Origin', 'Destination']]
df7_more_delayed_routes = df7.groupby(['Origin', 'Destination']).aggregate(sum)
df7_more_delayed_routes.sort_values(by=['Arrival_Delay'], ascending=False)[:5]

,,Arrival_Delay
Origin,Destination,
ORD,LGA,27346.559242
LGA,ORD,25737.194905
LAX,SFO,24652.000000
ORD,EWR,23937.000000
EWR,ORD,20235.389811


In [1453]:
#------- EXERCICI 2 --- ACABAT
### Aporta aquello que consideres relevante.
df['Total_Flights'] = df['Tail_Number'].value_counts().reset_index(drop=True)
df8 = df[['Tail_Number', 'Arrival_Delay', 'Departure_Delay', 'Air_Time', 'Total_Flights']]
df8_planes_more_delayed = df8.groupby('Tail_Number').aggregate(sum)
df8_planes_more_delayed.sort_values(by='Arrival_Delay', ascending=False)[:10]

,Arrival_Delay,Departure_Delay,Air_Time,Total_Flights
Tail_Number,,,,
N154SF,6976.000000,6820.0,4310.000000,36.0
N75995,6897.000000,7173.0,5706.000000,0.0
N571ML,6487.000000,6500.0,4524.000000,32.0
N505MJ,6129.000000,6282.0,6965.000000,227.0
N75992,6056.000000,6004.0,4676.000000,43.0
N511MJ,5914.000000,6028.0,7345.000000,40.0
N17175,5779.194905,5549.0,4841.408897,14.0
N521LR,5758.194905,5971.0,6844.408897,45.0
N75996,5748.000000,5687.0,3961.000000,90.0


En esta última consulta quería ver si había algún tipo de relación entre el avión que realiza el vuelo (Tail_Number), el retraso total y el número de vuelos realizado, pero la columna no muestra correctamente la información, me parece que porque no está correctamente indexada.

En caso de que estuviera correctamente indexada, puede que la base de datos describa una relación. Un siguiente paso podría ser el de investigar la causa de los retrasos de cada uno de los aviones ('Tail_Numnber') de esta última lista.

In [1454]:
df['Total_Flights'].max()

110.0

In [1455]:
df['Tail_Number'].value_counts()

N367SW    110
N608SW    109
N631SW    109
N347SW    108
N676SW    106
         ... 
N121UA      1
N171DZ      1
N802NW      1
N7BCAA      1
N1608       1
Name: Tail_Number, Length: 5272, dtype: int64

In [1456]:
#------- EXERCICI 3 --- ACABAT

#df.to_excel('Sprint_03_Tasca_02.xlsx', sheet_name='Sheet1')